In [1]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, "src", "utils")
sys.path.append(utils_dir)

In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from process_data import process_data
from weighted_accuracy import weighted_accuracy_scorer

In [3]:
X_train_path= os.path.join("..", "data","original_input", "X_train_filled.csv")
X_train_preprocessed = pd.read_csv(X_train_path, delimiter=',')
X_test_path= os.path.join("..", "data","original_input", "X_test_GgyECq8.csv")
X_test_preprocessed = pd.read_csv(X_test_path, delimiter=',')
y_train_path= os.path.join("..", "data","original_input", "y_train_jJtXgMX.csv")
y_train_preprocessed = pd.read_csv(y_train_path, delimiter=',')

delivery_start = X_test_preprocessed["DELIVERY_START"]

X = pd.concat([X_train_preprocessed, X_test_preprocessed])

In [5]:
X = process_data(X.copy(deep=True), "predicted_spot_price", None, "standard")
X

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std
DELIVERY_START,,,,,,,,
2022-01-01 01:00:00+00:00,0.010517,1.416173,0.287523,1.238146,-0.400271,-0.741471,-0.370537,-0.59073
2022-01-01 02:00:00+00:00,-0.258962,1.416173,0.287523,1.238146,-0.365879,-0.741471,-0.494713,-0.59073
2022-01-01 03:00:00+00:00,-0.383486,1.416173,0.287523,1.238146,-0.319703,-0.741471,-0.618985,-0.59073
2022-01-01 04:00:00+00:00,-0.418367,1.416173,0.287523,1.238146,-0.269069,-0.741471,-0.677004,-0.59073
2022-01-01 05:00:00+00:00,-0.371890,1.416173,0.287523,1.238146,-0.195188,-0.741471,-0.713568,-0.59073
...,...,...,...,...,...,...,...,...
2023-10-24 17:00:00+00:00,0.033249,-0.912866,0.821825,1.064708,0.193962,-0.741471,0.828013,-0.59073
2023-10-24 18:00:00+00:00,0.374792,-0.912866,0.821825,1.064708,0.411466,-0.741471,1.505366,-0.59073
2023-10-24 19:00:00+00:00,0.116081,-0.912866,0.821825,1.064708,0.641389,-0.741471,2.419943,-0.59073


In [6]:


split_date = pd.Timestamp('2023-04-01', tz='UTC')
X_train = X.loc[X.index < split_date]
X_test = X.loc[X.index >= split_date]
y_train = process_data(y_train_preprocessed.copy(deep=True), None, None, None)

In [7]:
lr = LinearRegression()

# Initialize Sequential Feature Selector
sfs = SequentialFeatureSelector(lr, n_features_to_select='auto', direction='forward', scoring='neg_mean_squared_error', cv=5)

# Fit SFS on the training data
sfs.fit(X_train, y_train)

# Get the support mask of selected features
selected_features = sfs.get_support()

# Names/indices of the selected features
selected_indices = [i for i, x in enumerate(selected_features) if x]
print("Selected feature indices:", selected_indices)

# Evaluate model performance with selected features
X_train_selected = X_train.loc[:, selected_features]
X_test_selected = X_test.loc[:, selected_features]


lr.fit(X_train_selected, y_train)
y_pred = lr.predict(X_test_selected)
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_test_result_LR7.csv", index = False)

Selected feature indices: [0, 4, 6, 7]


In [10]:
from sktime.forecasting.compose import make_reduction
from sklearn.linear_model import LinearRegression
from sktime.forecasting.base import ForecastingHorizon

# Extract best parameters from grid search
#best_params = 
fh = ForecastingHorizon(np.arange(1, len(X_test) + 1), is_relative=True)

# Since 'best_params_' includes keys with the 'estimator__' prefix for any estimator-specific parameters
# and 'window_length' without any prefix, we need to separate these when reconstructing the forecaster

# Initialize the estimator with the best parameters specific to LinearRegression
# We need to filter out and apply only those parameters that are specific to the estimator
#estimator_best_params = {key.replace("estimator__", ""): value for key, value in best_params.items() if key.startswith("estimator__")}
#linear_regressor = LinearRegression(**estimator_best_params)
linear_regressor = LinearRegression()

# Initialize the forecaster with the best window length and the newly configured estimator
# Note: 'strategy' is not typically a tuned parameter in this setup, so it's reused directly
forecaster_with_best_params = make_reduction(
    estimator=linear_regressor,
    #window_length=best_params["window_length"],
    strategy="recursive"
)

# Now, you can fit the forecaster with best parameters on your training data again if needed
# This step is optional if you're just setting up the forecaster for documentation or clarity
forecaster_with_best_params.fit(y_train, fh=fh, X=X_train)

# And use it for making predictions
y_pred_with_best_params = forecaster_with_best_params.predict(fh=fh, X=X_test)

TypeError: unsupported operand type(s) for *=: 'int' and 'NoneType'

In [ ]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred_with_best_params.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../data/output_y_test/y_test_result_LR9.csv", index = False)